In [ ]:
import numpy as np

In [ ]:
# 활성화 함수
class activation_function:
  # 시그모이드 함수
  def sigmoid(self, x):
    return 1 / (1+np.e**-x)
  
  # 시그모이드 함수의 미분 함수
  def sigmoid_diff(self, x):  
    return self.sigmoid(x) * (1 - self.sigmoid(x))

In [ ]:
# 비용 함수 클래스, 비용 함수의 종류에 따라 다른 클래스 내 함수를 사용한다.
class cost_function:
  # 예측값
  predict = []
  # 타겟값
  target = []
  # 비용 함수값
  error_cost = []

  # 오차 제곱합
  def errer_squared_sum(self, predict, target):
    self.predict = predict
    self.target = target

    self.error_cost = np.sum(0.5*((predict - target)**2))
    return self.error_cost

  # 오차 제곱합 미분 함수
  def diff_error_squared_sum(self, predict, target):
    self.predict = predict
    self.target = target

    return self.predict - self.target

In [ ]:
class Rnn():
  

In [ ]:
# Sequential 클래스
class Sequential():
  # 층을 차례대로 쌓아서 구성, 각 층을 배열 형태로 지정
  layer = []

  # 각 레이어 별 출력값
  layer_result = []

  # compile 정보가 저장되어 있는 딕셔너리 객체
  compile_dir = []

  # fit, 훈련 정보가 저장되어 있는 객체
  fit_dir = []

  # dense
  dense = Dense()

  # Rnn
  rnn = Rnn()

  # 비용 함수
  cost = cost_function()

  # 활성화 함수
  activation = activation_function()

  # 비용 함수 값
  cost_result = []

  # 각 층별 가중치
  weight_layer = []

  # 각 층별 delta 값
  delta_layer = []

  # 가중치 업데이트 크기
  weight_update_arr = []

  # 각 층을 쌓기 위한 함수
  def add(self, dict):
    self.layer.append(dict)

  # fit 함수
  def fit(self, dict):
    self.fit = dict

    self.train(dict['data'], dict['target'])
    
  # compile 함수
  def compile(self, loss = "mse", metrics = ['accuaray']):
    compile_dict = dict()
    compile_dict['loss'] = loss
    compile_dict['metrics'] = metrics
    
    return compile_dict
    
  # train 함수
  def train(self, data, target):
    for i in range(len(self.layer) - 1):
      layer_type = self.layer[i].get('layer_type')
      if(layer_type == 'Dense'):
        self.dense.train(self.layer[i])

      if(layer_type == 'Rnn'):
        self.rnn.train(self.layer[i])

    # 비용 함수 계산, 예측값 마지막 노드의 출력
    predict = self.dense.node_output[-1]

    # 비용 함수 값의 계산
    self.cost_result = self.cost.errer_squared_sum(predict, target)

    # delta 값의 계산
    self.cal_delta(target)

  # delta 값 계산,
  def cal_delta(self, target):
    
    # 초기 delta, 비용 함수의 미분값을 사용한다.
    delta = self.cost.diff_error_squared_sum(self.dense.node_output[-1], target)

    # 활성화 함수 적용 전, 노드 입력값의 변화량에 대한 비용 함수의 변화량
    # 해당 층의 활성화 함수의 미분 함수와의 연산 수행
    if(self.layer[-1].get('activation') == 'sigmoid'):
      delta = delta * self.activation.sigmoid_diff(self.dense.node_input[-1])

    # 해당 delta 값 저장
    self.delta_layer.append(delta)

    # 남은 레이어에 대해 delta 값 계산
    for i in range(len(self.layer) - 1):
      # 이전 층의 노드 출력의 변화량에 대한 비용함수의 변화량은 가중치에 해당한다.
      delta = self.weight.T @ delta

      # 해당 층의 활성화 함수의 미분 함수와의 연산 수행
      if(self.layer[-1 -i].get('activation') == 'sigmoid'): 
        delta = delta * self.activation.sigmoid_diff(self.dense.node_input[-1 - i])
    
      # 해당 delta 값 저장
      self.delta_layer.append(delta)

  
  # 가중치 업데이트량 크기 계산
  def weight_update(self):
    # 연산의 편리함을 위해 뒤집어준다.
    self.delta_layer = self.delta_layer[::-1]

    for i in range(len(self.weight)):
      self.weight_update_arr.append(self.delta_layr[i] @ self.node_output[i].T)

      self.weight[i] = self.weight[i] - self.weight_update_arr[i].T